In [1]:
import os

import pandas as pd

""" 
2/2/23 - All methods realted to dPCA here.
Thought of consoldiating into functions, but best to leave here, and isntaed I tried to carefully doc below.

Script: analy_dpca_script_quick.py. It is 

Building from
_snippets_rsa.ipynb.
"""

' \n2/2/23 - All methods realted to dPCA here.\nThought of consoldiating into functions, but best to leave here, and isntaed I tried to carefully doc below.\n\nScript: analy_dpca_script_quick.py. It is \n\nBuilding from\n_snippets_rsa.ipynb.\n'

# DEVO - working with single PA

In [2]:

def dpca_compute_pa_to_space(pa, effect_vars, keep_all_margs=True):
    from neuralmonkey.scripts.analy_dpca_script_quick import preprocess_pa_to_frtensor

    from dPCA import dPCA
    # Clean up PA

    # Preprocess
    R, trialR, map_var_to_lev, map_grp_to_idx, params_dpca, PAnorm = preprocess_pa_to_frtensor(pa, effect_vars, keep_all_margs=keep_all_margs)
    
    print("---- OUTPUT")
    print("R.shape:", R.shape)
    print("trialR:", trialR.shape)
    print("map_var_to_lev:", map_var_to_lev)
    print("map_grp_to_idx:", map_grp_to_idx)
    print("params_dpca:", params_dpca)

    #### Fit model

    # We then instantiate a dPCA model where the two parameter axis are labeled by 's' (stimulus) and 't' (time) respectively. We set regularizer to 'auto' to optimize the regularization parameter when we fit the data.
    labels = params_dpca["labels"]
    join = params_dpca["join"]
    n_components = params_dpca["n_components"]
    dpca = dPCA.dPCA(labels=labels, regularizer='auto', join=join, n_components=n_components)
    dpca.protect = ['t']
        
    # Now fit the data (R) using the model we just instatiated. Note that we only need trial-to-trial data when we want to optimize over the regularization parameter.
    Z = dpca.fit_transform(R,trialR)
    
    return dpca, Z, R, trialR, map_var_to_lev, map_grp_to_idx, params_dpca, PAnorm
# 
# effect_vars = ["shape"]
# dpca_compute_pa_to_space(pa, effect_vars)

In [3]:
# 

# Analysis approach 1 - separate dPCA for each PA

In [ ]:
from neuralmonkey.scripts.analy_dpca_script_quick import preprocess_pa_to_frtensor
from neuralmonkey.classes.snippets import load_and_concat_mult_snippets
from neuralmonkey.classes.session import load_mult_session_helper
import os
import pandas as pd
from neuralmonkey.analyses.state_space_good import snippets_extract_popanals_split_bregion_twind


In [ ]:
from pythonlib.tools.plottools import savefig
from pythonlib.globals import PATH_ANALYSIS_OUTCOMES
import os
import sys

SAVEDIR_ANALYSES = f"{PATH_ANALYSIS_OUTCOMES}/recordings/main/dPCA"

############### PARAMS
animal = "Diego"
date = 230616
exclude_bad_areas = True
SPIKES_VERSION = "tdt" # since Snippets not yet extracted for ks
bin_by_time_dur = 0.05
bin_by_time_slide = 0.025
which_level = "substroke"

if False:
    # METHOD 2 - Merging across time windows into single PA bofre doing dPCA
    question = "SP_shape_loc_TIME"
    slice_agg_slices = [
        ("trial", "03_samp", (-0.3, 0.5)),
        ("trial", "04_go_cue", (-0.45, 0.25)),
        ("trial", "06_on_strokeidx_0", (-0.25, 0.7))
    ]
    slice_agg_vars_to_split = ["bregion"]
    slice_agg_concat_dim = "times"
    
    list_time_windows = [sl[2] for sl in slice_agg_slices]
    events_keep = list(set([sl[1] for sl in slice_agg_slices]))
    print(list_time_windows)
else:
    # METHOD 1 - Standard, running separately for each PA
    question = "SS_shape"
    slice_agg_slices = None
    slice_agg_vars_to_split = None
    slice_agg_concat_dim = None
    
    list_time_windows = [(-0.3, 0.)]
    events_keep = ["00_substrk"]
    # list_time_windows = [(-0.3, 0.)]
    # events_keep = ["06_on_strokeidx_0"]
    print(list_time_windows)



In [ ]:
# Load q_params
from neuralmonkey.analyses.rsa import rsagood_questions_dict, rsagood_questions_params
q_params = rsagood_questions_dict(animal, date, question)[question]

In [ ]:
# Load data
from neuralmonkey.classes.population_mult import dfallpa_extraction_load_wrapper

DFallpa = dfallpa_extraction_load_wrapper(animal, date, question, list_time_windows,
                                          which_level=which_level,
                                bin_by_time_dur = bin_by_time_dur, 
                                bin_by_time_slide = bin_by_time_slide)


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from neuralmonkey.scripts.analy_dpca_script_quick import plot_statespace_2d_overlaying_all_othervar

keep_all_margs = True
PLOT = True

In [ ]:
if False:
    # dPCA only works if have data for all conjucntions... Check if that's the case.
    from pythonlib.tools.pandastools import conjunction_vars_prune_to_balance
    
    df = pa.Xlabels["trials"]
    conjunction_vars_prune_to_balance(df, effect_vars[0], effect_vars[1], True)


In [ ]:
# Get results for a single pa
from neuralmonkey.scripts.analy_dpca_script_quick import plothelper_get_variables


In [ ]:

if True:
    # Overwrite effect var
    effect_vars = ["shape"]
    q_params["effect_vars"] = effect_vars
else:
    effect_vars = q_params["effect_vars"]
        
PLOT = False

##### savedir
from pythonlib.tools.listtools import stringify_list
a = stringify_list(list_time_windows, return_as_str=True)
b = stringify_list(events_keep, return_as_str=True)

SAVEDIR = f"{SAVEDIR_ANALYSES}/{animal}-{date}/{question}/{a}--{b}--{keep_all_margs}"

RES = []
for i, row in DFallpa.iterrows():

    pa = row["pa"]
    br = row["bregion"]
    wl = row["which_level"]
    ev = row["event"]
    tw = row["twind"]


    # Clean up PA
    from neuralmonkey.analyses.rsa import preprocess_rsa_prepare_popanal_wrapper
    pa, res_check_tasksets, res_check_effectvars = preprocess_rsa_prepare_popanal_wrapper(pa, **q_params)
    
    # Compute dPCs, and project data.
    dpca, Z, R, trialR, map_var_to_lev, map_grp_to_idx, params_dpca, panorm = dpca_compute_pa_to_space(pa, effect_vars, keep_all_margs=keep_all_margs)
    plothelper_get_variables(Z, effect_vars, params_dpca) # Add variables to params
        
    # Znew = dpca.transform(R)
    # for k in Znew.keys():
    #     assert np.all(Z[k] == Znew[k]) 

    if PLOT:
    
        savedir = f"{SAVEDIR}/each_pa/{wl}-{ev}-{br}-{tw}"
        os.makedirs(savedir, exist_ok=True)
        print(" *** Saving to:", savedir)
        
        from pythonlib.tools.expttools import writeDictToTxt, writeDictToYaml
        path = f"{savedir}/res_check_effectvars.txt"
        writeDictToTxt(res_check_effectvars, path)
        path = f"{savedir}/res_check_tasksets.txt"
        writeDictToTxt(res_check_tasksets, path)
        
        params_dpca["data_shape-trial_N_features_time"] = trialR.shape
        writeDictToYaml(params_dpca, f"{savedir}/params.yaml")
    
        plot_all_results_single(dpca, Z, effect_vars, params_dpca, savedir)


    RES.append({
        "dpca":dpca,
        "bregion":br,
        "which_level":wl,
        "event":ev,
        "twind":tw,
        "explained_var":dpca.explained_variance_ratio_,
        "marginalizations":list(dpca.marginalizations.keys()),
        "params_dpca":params_dpca,
        "map_var_to_lev":map_var_to_lev,
        "map_grp_to_idx":map_grp_to_idx,
        # "Z":Z,
        "R":R,
        "trialR":trialR,        
        "times":pa.Times,
        "panorm":panorm
    })

assert False
# Dont conver to df, too slow
# DFRES = pd.DataFrame(RES)

if PLOT:
    from neuralmonkey.scripts.analy_dpca_script_quick import plot_all_results_mult
    plot_all_results_mult(DFRES, SAVEDIR)


In [ ]:
## IGNORE THIS
from neuralmonkey.scripts.analy_dpca_script_quick import plot_all_results_single

savedir = f"{SAVEDIR}/each_pa/{wl}-{ev}-{br}-{tw}"
os.makedirs(savedir, exist_ok=True)
print(" *** Saving to:", savedir)

from pythonlib.tools.expttools import writeDictToTxt, writeDictToYaml
path = f"{savedir}/res_check_effectvars.txt"
writeDictToTxt(res_check_effectvars, path)
path = f"{savedir}/res_check_tasksets.txt"
writeDictToTxt(res_check_tasksets, path)

params_dpca["data_shape-trial_N_features_time"] = trialR.shape
writeDictToYaml(params_dpca, f"{savedir}/params.yaml")

plot_all_results_single(dpca, Z, effect_vars, params_dpca, savedir)


### Try out on single pa

In [11]:
from pythonlib.tools.plottools import savefig
from pythonlib.globals import PATH_ANALYSIS_OUTCOMES
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from neuralmonkey.scripts.analy_dpca_script_quick import plot_statespace_2d_overlaying_all_othervar

# Get results for a single pa
from neuralmonkey.scripts.analy_dpca_script_quick import plothelper_get_variables
from neuralmonkey.analyses.state_space_good import trajgood_plot_colorby_splotby_scalar, trajgood_plot_colorby_splotby_scalar_helper

from pythonlib.tools.plottools import savefig
from pythonlib.tools.pandastools import append_col_with_grp_index
from neuralmonkey.analyses.rsa import preprocess_rsa_prepare_popanal_wrapper

from pythonlib.tools.listtools import stringify_list

# Load q_params
from neuralmonkey.analyses.rsa import rsagood_questions_dict, rsagood_questions_params
# Load data
from neuralmonkey.classes.population_mult import dfallpa_extraction_load_wrapper

SAVEDIR_ANALYSES = f"{PATH_ANALYSIS_OUTCOMES}/recordings/main/dPCA"
keep_all_margs = False

############### PARAMS
# animal = "Diego"
# dates = [230616, 230618, 230619]
animal = "Pancho"
dates = [220716, 220715, 220717, 220718, 220719]

list_failures =[]
list_errs = []
for date in dates:
    try:
        exclude_bad_areas = True
        SPIKES_VERSION = "tdt" # since Snippets not yet extracted for ks
        bin_by_time_dur = 0.05
        bin_by_time_slide = 0.025
        which_level = "substroke"
        
        if False:
            # METHOD 2 - Merging across time windows into single PA bofre doing dPCA
            question = "SP_shape_loc_TIME"
            slice_agg_slices = [
                ("trial", "03_samp", (-0.3, 0.5)),
                ("trial", "04_go_cue", (-0.45, 0.25)),
                ("trial", "06_on_strokeidx_0", (-0.25, 0.7))
            ]
            slice_agg_vars_to_split = ["bregion"]
            slice_agg_concat_dim = "times"
            
            list_time_windows = [sl[2] for sl in slice_agg_slices]
            events_keep = list(set([sl[1] for sl in slice_agg_slices]))
            print(list_time_windows)
        else:
            # METHOD 1 - Standard, running separately for each PA
            question = "SS_shape"
            slice_agg_slices = None
            slice_agg_vars_to_split = None
            slice_agg_concat_dim = None
            
            list_time_windows = [(-0.3, 0.)]
            events_keep = ["00_substrk"]
            # list_time_windows = [(-0.3, 0.)]
            # events_keep = ["06_on_strokeidx_0"]
            print(list_time_windows)
        
        for list_time_windows in [
            [(-0.2, 0.)],
            [(0., 0.2)]
            ]:
            q_params = rsagood_questions_dict(animal, date, question)[question]
            
            DFallpa = dfallpa_extraction_load_wrapper(animal, date, question, list_time_windows,
                                                      which_level=which_level,
                                            bin_by_time_dur = bin_by_time_dur, 
                                            bin_by_time_slide = bin_by_time_slide)
            
            
            
            a = stringify_list(list_time_windows, return_as_str=True)
            b = stringify_list(events_keep, return_as_str=True)
            
            SAVEDIR = f"{SAVEDIR_ANALYSES}/{animal}-{date}/{question}/{a}--{b}--{keep_all_margs}"
            
            list_br = DFallpa["bregion"].unique().tolist()
            # effect_vars = ["seqc_0_shape", "seqc_0_loc"]
            # br = "PMv"
            # ev = "06_on_strokeidx_0"
            # br = "PMv"
            ev = "00_substrk"
            
            for br in list_br:
                tmp = DFallpa[(DFallpa["bregion"]==br) & (DFallpa["event"]==ev)]
                assert len(tmp)==1
                pa = tmp["pa"].values[0]
                
                pa.Xlabels["trials"] = append_col_with_grp_index(pa.Xlabels["trials"], ["shape", "index_within_stroke"], "shape_idx", False)
                pa.Xlabels["trials"] = append_col_with_grp_index(pa.Xlabels["trials"], ["distcum_binned", "angle_binned", "circularity_binned"], "di_an_ci_binned", False)
            
                # Clean up PA
                pa, res_check_tasksets, res_check_effectvars = preprocess_rsa_prepare_popanal_wrapper(pa, **q_params)
                
                # Restrict analysis to just first substroke
                if True:
                    pa = pa.slice_by_labels("trials", "index_within_stroke", [0])
                
                for effect_vars in [
                    ["shape"],
                    ["dist_angle"]
                ]:
                    
                    if effect_vars == ["shape"]:
                        marginalization="s"
                    elif effect_vars == ["dist_angle"]:
                        marginalization="m"
                    else:
                        assert False
            
                    savedir = f"{SAVEDIR}/scatter_color_diff_ways/effect_vars-{'|'.join(effect_vars)}"
                    os.makedirs(savedir, exist_ok=True)
            
                    # Compute dPCs, and project data.
                    dpca, Z, R, trialR, map_var_to_lev, map_grp_to_idx, params_dpca, panorm = dpca_compute_pa_to_space(pa, effect_vars, keep_all_margs=keep_all_margs)
                    plothelper_get_variables(Z, effect_vars, params_dpca) # Add variables to params
                    
                    ##### Single trial analysis
                    def transform_from_pa(dpca, pa, marginalization):
                        """
                        
                        :param dpca: 
                        :param pa: (nchans, ntrials, ntimes)
                        :param marginalization: 
                        :return: 
                        - trialX_proj, (ndims_reduced, ntrials, ntimes)
                        """
                        
                        if False:
                            # NOT WORKING!!
                            # Vectorized
                            trialX = np.transpose(pa.X, [1, 0, 2]) # (trials, chans, times)
                            trialX_proj = transform_trial(dpca, trialX, "s")
                        else:
                            # Debuggin, Redo it without using trialR
                            D = dpca.D[marginalization] # (nchan, ndim)
                            
                            outs = []
                            for trial in range(pa.X.shape[1]):
                                x = pa.X[:,trial,:] # (nchans, ntimes)
                                
                                x_proj = np.dot(D.T, x) # (ndim, ntimes)
                                
                                outs.append(x_proj)
                                
                            trialX_proj = np.stack(outs) # (ntrials, ndims, ntimes)
                        assert not np.any(np.isnan(trialX_proj))
                    
                        return trialX_proj
                        
                    def transform_trial(dpca, X, marginalization):
                        """
                    
                        :param dpca: 
                        :param X: shape (ntrials, nchans, ...) 
                        :param marginalization: 
                        :return: 
                        """
                        
                        assert not np.any(np.isnan(X))
                        # print(X.shape) # (trials, olddim(i.e., chans), features)
                    
                        axis_chans = 1
                        D, Xr         = dpca.D[marginalization], X.reshape((X.shape[axis_chans],-1))
                        
                        print(D.shape)
                        print(Xr.shape)
                        newshape = tuple((X.shape[0], D.shape[1]) + X.shape[2:])# (trials, newdim, features)
                        print(newshape)
                        X_transformed = np.dot(D.T, Xr).reshape(newshape)
                        
                        # print(X_transformed.shape)
                        # print(X.shape)
                        assert len(X_transformed.shape)==len(X.shape)
                        assert X_transformed.shape[0] == X.shape[0]
                        assert X_transformed.shape[2:] == X.shape[2:]
                        
                        return X_transformed
                    # First, convert to final data using PA (e.g., scalar)
                    pathis = panorm.agg_wrapper("times")
                    
                    dflab = pathis.Xlabels["trials"]
                    
                    trialX_proj = transform_from_pa(dpca, pathis, marginalization)
                    dim1 = 0
                    dim2 = 1
                    xs = trialX_proj[:, dim1]
                    ys = trialX_proj[:, dim2]
                
                    for color_var in ["shape", "di_an_ci_binned", "angle_binned"]:
                        for subplot_var in ["bregion", "shape", "di_an_ci_binned", "angle_binned"]:
                            # color_var = "seqc_0_shape"
                            # subplot_var = "seqc_0_loc"
                            # color_var = "shape"
                            # subplot_var = "bregion"
                            
                            # color_var = "di_an_ci_binned"
                            # subplot_var = "bregion"
                            
                            labels = dflab[color_var]
                            
                            
                            fig, axes = trajgood_plot_colorby_splotby_scalar_helper(xs, ys, labels, dflab[subplot_var],
                                                                        color_var, subplot_var, overlay_mean=False, SIZE=7)
                            savefig(fig, f"{savedir}/{br}-color_{color_var}-splot_{subplot_var}.pdf")
                            
                            plt.close("all")
    except Exception as err:
        list_failures.append(date)
        list_errs.append(err)
        pass

[(-0.3, 0.0)]
Searching using this string:
/mnt/Freiwald/ltian/recordings/*Pancho*/*220716*/**
Found this many paths:
2
---
/mnt/Freiwald/ltian/recordings/Pancho/220716/Pancho-220716-144103
---
/mnt/Freiwald/ltian/recordings/Pancho/220716/Pancho-220716-150716
session:  0
Searching using this string:
/mnt/Freiwald/ltian/recordings/*Pancho*/*220716*/**
Found this many paths:
2
---
/mnt/Freiwald/ltian/recordings/Pancho/220716/Pancho-220716-144103
---
/mnt/Freiwald/ltian/recordings/Pancho/220716/Pancho-220716-150716
Beh Sessions hand netered (mapping: rec sess --> beh sess):  [1, 3]
Beh Sessions that exist on this date:  {220716: [(1, 'priminvar3b'), (3, 'priminvar3b')]}
------------------------------
Loading this neural session: 0
Loading these beh expts: ['priminvar3b']
Loading these beh sessions: [1]
Using this beh_trial_map_list: [(1, 0)]
Searching using this string:
/mnt/Freiwald/ltian/recordings/*Pancho*/*220716*/**
Found this many paths:
2
---
/mnt/Freiwald/ltian/recordings/Pancho/2

In [14]:
list_errs

[FileNotFoundError(2, 'No such file or directory')]

In [ ]:
# Color by binned stroke features
pa.Xlabels["trials"][:2]

In [ ]:
# convert to scalar
timebin = 0

# APproach 2: Compare two datasets PA train vs. test (each across all areas)

In [ ]:
from pythonlib.tools.plottools import savefig
from pythonlib.globals import PATH_ANALYSIS_OUTCOMES
import os
import sys

SAVEDIR_ANALYSES = f"{PATH_ANALYSIS_OUTCOMES}/recordings/main/dPCA"

############### PARAMS
animal = "Diego"
date = 230615
exclude_bad_areas = True
SPIKES_VERSION = "tdt" # since Snippets not yet extracted for ks
bin_by_time_dur = 0.05
bin_by_time_slide = 0.025

if True:
    # Train/test using different time windows. The PA related stuff is same as standard approach.
    question = "SP_shape_loc"
    slice_agg_slices = None
    slice_agg_vars_to_split = None
    slice_agg_concat_dim = None
    
    # list_time_windows = [(0.2, 0.6), (-0.3, 0.3)]
    list_time_windows = [(0.1, 0.3), (-0.3, 0.3)] # Immediate visual response
    events_keep = ["03_samp", "06_on_strokeidx_0"]
    print(list_time_windows)



In [ ]:
# Load data
from neuralmonkey.classes.population_mult import dfallpa_extraction_load_wrapper

DFallpa = dfallpa_extraction_load_wrapper(animal, date, question, list_time_windows,
                                bin_by_time_dur = bin_by_time_dur, 
                                bin_by_time_slide = bin_by_time_slide)


In [ ]:
effect_vars = q_params["effect_vars"]
print("Effect vars:", effect_vars)
keep_all_margs = True

In [ ]:
from neuralmonkey.scripts.analy_dpca_script_quick import plothelper_get_variables
from neuralmonkey.scripts.analy_dpca_script_quick import plot_all_results_single
list_br = DFallpa["bregion"].unique().tolist()

for keep_all_margs in [True, False]:
    wl = "trial"
    ev_train = events_keep[0]
    tw_train = list_time_windows[0]
    
    ev_test = events_keep[1]
    tw_test = list_time_windows[1]
    
    from pythonlib.tools.expttools import writeDictToTxt
    SAVEDIR = f"{SAVEDIR_ANALYSES}/{animal}-{date}/{question}/train_test_diff_evtw/TRAIN-{ev_train}-{tw_train}-TEST-{ev_test}-{tw_test}-keepallmarg_{keep_all_margs}"
    os.makedirs(SAVEDIR, exist_ok=True)
    params = {}
    params["wl"] = wl
    params["ev_train"] = ev_train
    params["tw_train"] = tw_train
    params["ev_test"] = ev_test
    params["tw_test"] = tw_test
    writeDictToTxt(params, f"{SAVEDIR}/params.txt")
    
    ## Fit PCs and evaluate test data.
    RES_TRAIN = []
    RES_TEST = []
    for br in list_br:
        a = DFallpa["event"] == ev_train
        b = DFallpa["twind"] == tw_train
        c = DFallpa["bregion"] == br
        
        tmp =DFallpa[(a & b & c)]
        assert len(tmp)==1
        pa_train = tmp["pa"].values[0]
    
        a = DFallpa["event"] == ev_test
        b = DFallpa["twind"] == tw_test
        tmp =DFallpa[(a & b & c)]
        assert len(tmp)==1
        pa_test = tmp["pa"].values[0]
    
        # Train
        from dPCA import dPCA
        R, trialR, map_var_to_lev, map_grp_to_idx, params_dpca, PAnorm = preprocess_pa_to_frtensor(pa_train, effect_vars, keep_all_margs=keep_all_margs)
        
        labels = params_dpca["labels"]
        join = params_dpca["join"]
        n_components = params_dpca["n_components"]
        dpca = dPCA.dPCA(labels=labels, regularizer='auto', join=join, n_components=n_components)
        dpca.protect = ['t']
        
        # Now fit the data (R) using the model we just instatiated. Note that we only need trial-to-trial data when we want to optimize over the regularization parameter.
        Z = dpca.fit_transform(R,trialR)
            
        ############ PLOT TRAINING DATA
        savedir = f"{SAVEDIR}/each_bregion/{br}/figures_summary_TRAIN"
        os.makedirs(savedir, exist_ok=True)
        # Test, by projecting new data onto this space
        R, trialR, _, _, params_dpca, PAnorm = preprocess_pa_to_frtensor(pa_train, effect_vars, keep_all_margs=keep_all_margs)
        Ztrain = dpca.transform(R)
        plot_all_results_single(dpca, Ztrain, effect_vars, params_dpca, savedir)    
        plt.close("all")
    
        RES_TRAIN.append({
            "bregion":br,
            "which_level":wl,
            "event":ev_train,
            "twind":tw_train,
            "explained_var":dpca.explained_variance_ratio_,
            "marginalizations":list(dpca.marginalizations.keys()),
            "params_dpca":params_dpca,
            "map_var_to_lev":map_var_to_lev,
            "map_grp_to_idx":map_grp_to_idx,
            "Z":Ztrain
        })
    
        ############ PLOT TESTING DATA
        savedir = f"{SAVEDIR}/each_bregion/{br}/figures_summary_TEST"
        os.makedirs(savedir, exist_ok=True)
        # Test, by projecting new data onto this space
        R, trialR, _, _, params_dpca, PAnorm = preprocess_pa_to_frtensor(pa_test, effect_vars, keep_all_margs=keep_all_margs)
        Ztest = dpca.transform(R)
        plot_all_results_single(dpca, Ztest, effect_vars, params_dpca, savedir)    
        plt.close("all")
                
        RES_TEST.append({
            "bregion":br,
            "which_level":wl,
            "event":ev_test,
            "twind":tw_test,
            "explained_var":dpca.explained_variance_ratio_,
            "marginalizations":list(dpca.marginalizations.keys()),
            "params_dpca":params_dpca,
            "map_var_to_lev":map_var_to_lev,
            "map_grp_to_idx":map_grp_to_idx,
            "Z":Ztest
        })
        
        plt.close("all")
        
        #### PLOT SUMMARY for test and train.
        from neuralmonkey.scripts.analy_dpca_script_quick import plot_all_results_mult
        
        DFRES_TRAIN = pd.DataFrame(RES_TRAIN)
        savedir = f"{SAVEDIR}/train"
        os.makedirs(savedir, exist_ok=True)
        plot_all_results_mult(DFRES_TRAIN, savedir)
        plt.close("all")
        
        DFRES_TEST = pd.DataFrame(RES_TEST)
        savedir = f"{SAVEDIR}/test"
        os.makedirs(savedir, exist_ok=True)
        plot_all_results_mult(DFRES_TEST, savedir)
        plt.close("all")
                
                
    ##### SUMMARY: Compare similarity of activity of test data, projected onto old subspace
    # PARAMS
    # - Time bins, indices into z[:,:, :, idx]
    tbin_train = -1
    tbin_test = 0
    version_distance = "_pearson_raw"
    PLOT_EACH_HEATMAP=False
    
    # RUN
    savedir = f"{SAVEDIR}/RSA"
    os.makedirs(savedir, exist_ok=True)
    from neuralmonkey.scripts.analy_dpca_script_quick import plothelper_get_variables
    
    ts, margs, map_var_to_idx, map_margstr_to_var, map_var_lev_to_pcol, map_var_to_lev = plothelper_get_variables(Z, effect_vars, params_dpca)
    
    # RSA analysis.
    map_grp_to_idx = params_dpca["map_grp_to_idx"]
    map_var_to_idx = params_dpca["map_var_to_idx"]
    margs = params_dpca["margs"]
    map_traintest_to_Z = {
        "train":Ztrain,
        "test":Ztest
    }
    SCORES = []
    for marg in margs:
        map_bregion_Ztrain = {row["bregion"]:row["Z"][marg] for _, row in DFRES_TRAIN.iterrows()}
        map_bregion_Ztest = {row["bregion"]:row["Z"][marg] for _, row in DFRES_TEST.iterrows()} 
        
        list_br = list(map_bregion_Ztrain.keys())
        for br in list_br:    
            ######## Collect z across all variables, and train/test.
            tmp = []
            for train_test in ["train", "test"]:
                if train_test=="train":
                    tbin = tbin_train
                    Zthis = map_bregion_Ztrain[br]
                elif train_test=="test":
                    tbin = tbin_test                
                    Zthis = map_bregion_Ztest[br]
                else:
                    assert False
                    
                # Collect data across effect vars
                for grp, idx in map_grp_to_idx.items():
                    dat = {}
                    dat["bregion"] = br
                    for var, i in map_var_to_idx.items():
                        dat[var] = grp[i]    
                    dat["train_test"] = train_test
                    dat["z"] = Zthis[:, idx[0], idx[1], tbin]
                    tmp.append(dat)        
            dftmp = pd.DataFrame(tmp)
        
        
            ########### SCORE AND PLOT
            # Convert to Cluster
            from pythonlib.cluster.clustclass import Clusters
            X = np.stack(dftmp["z"]) # (ndat, ndims)
            label_vars = effect_vars + ["train_test"]
            labels = dftmp.loc[:, label_vars].values.tolist()
            labels = [tuple(l) for l in labels]
            labels_cols = list(range(X.shape[1]))
            Clraw = Clusters(X, labels, labels_cols, "rsa", {"label_vars":label_vars})
            Clsim = Clraw.distsimmat_convert(version_distance)
                
            if PLOT_EACH_HEATMAP:
                # plot 
                fig, ax = Clraw.rsa_plot_heatmap(sort_order=(0,1,2), diverge=True)
                savefig(fig, f"{savedir}/rawheat-marg_{marg}-{br}.pdf")
                
                fig, ax = Clsim.rsa_plot_heatmap(sort_order=(0,1,2))
                savefig(fig, f"{savedir}/rawdiff_pearson-marg_{marg}-{br}.pdf")
            
            # compute score
            res = []
            for var_effect in effect_vars:
                vars_all = label_vars
                var_other = [v for v in effect_vars if not v==var_effect][0]
                vars_test_invariance_over_dict = {
                    "same":[var_other],
                    "diff":["train_test"]
                }
                out = Clsim.rsa_distmat_score_same_diff(var_effect, vars_all, vars_test_invariance_over_dict, PLOT=False)
                
                SCORES.append({
                    "EffS_CtxS":out["EffS_CtxS"],
                    "EffD_CtxS":out["EffD_CtxS"],
                    "EffS_CtxD":out["EffS_CtxD"],
                    "EffD_CtxD":out["EffD_CtxD"],
                    "bregion":br,
                    "var_effect":var_effect,                
                    "var_other":var_other,
                    "vars_all":vars_all,
                    "marginalization":marg,
                })
                
    ######### PLOT
    dfscores = pd.DataFrame(SCORES)
    
    list_marg = dfscores["marginalization"].unique()
    import seaborn as sns
    
    ####
    for yvar in [
        "EffS_CtxS",
        "EffD_CtxS",
        "EffS_CtxD",
        "EffD_CtxD"]:
        
        ncols =2
        nrows = int(np.ceil(len(list_marg)/ncols))
        fig, axes = plt.subplots(nrows, ncols, figsize=(12,nrows*5), sharey=True)
        
        for marg, ax in zip(list_marg, axes.flatten()):
            dfthis = dfscores[dfscores["marginalization"]==marg]
            sns.barplot(data=dfthis, x="bregion", y=yvar, ax=ax, hue="var_effect")
            ax.set_title(marg)
        
        print("Plotting for:", yvar)
        savefig(fig, f"{savedir}/corr-test_vs_train-projected_on_train_space-{version_distance}-yvar_{yvar}.pdf")  
        plt.close("all")
    
    #### Plot summary of scores.   
    dfscores["score_CtxD"] = dfscores["EffS_CtxD"] - dfscores["EffD_CtxD"]
    fig = sns.catplot(data=dfscores, x="bregion", y="score_CtxD", hue="var_effect", col="marginalization", col_wrap=2, kind="bar")
    savefig(fig, f"{savedir}/score_EffS_CtxD_minus_EffD_CtxD.pdf")
    
    # --
    from pythonlib.tools.pandastools import unpivot
    value_vars = [
                "EffS_CtxS",
                "EffD_CtxS",
                "EffS_CtxD",
                "EffD_CtxD"]
    
    dfscores_long = unpivot(dfscores, ["bregion", "var_effect", "var_other", "marginalization", "vars_all"], value_vars, "score_kind", "score")
    dfscores_long
    score_kinds_keep = ["EffD_CtxS", "EffS_CtxD", "EffD_CtxD"]
    dfscores_long = dfscores_long[dfscores_long["score_kind"].isin(score_kinds_keep)].reset_index(drop=True)
    fig = sns.catplot(data=dfscores_long, x="bregion", y="score", hue="score_kind", col="var_effect", row="marginalization", kind="bar")
    savefig(fig, f"{savedir}/all_effects.pdf")
    plt.close("all")
    
    
##### A single plot overlaying train and test, where one time bin for train is concatted to one for test.
# [OLDER, beducase this is difficult to compare represntational structure of train vs. for test.

# Construct a new fake Z, which makes Ztrain and Ztest be the two "timepoints"
def _concat_ztrain_ztest(ztrain, ztest, tbin_train, tbin_test):
    """ Given two matrics of (a,b,c,n) and (a,b,c,m), takes one 
    time slice each, to output (a,b,c,2)
    """
    # tbin_train = -1
    # tbin_test = 0
    zfake = np.stack((ztrain[:,:,:,tbin_train], ztest[:,:,:,tbin_test]), axis=3)
    return zfake

savedir = f"{SAVEDIR}/state_space_combined"
os.makedirs(savedir, exist_ok=True)
# Note which time bins
from pythonlib.tools.expttools import writeDictToTxt
prms = {
    "tbin_train":tbin_train,
    "tbin_test":tbin_test,
}
writeDictToTxt(prms, f"{savedir}/prms_tbins.txt")

SAME_AXES_ACROSS_MARGS = False

for marg in margs:
    map_bregion_Ztrain = {row["bregion"]:row["Z"][marg] for _, row in DFRES_TRAIN.iterrows()}
    map_bregion_Ztest = {row["bregion"]:row["Z"][marg] for _, row in DFRES_TEST.iterrows()}     
    for br in list_br:
        ztrain = map_bregion_Ztrain[br]
        ztest = map_bregion_Ztest[br]
        var = map_margstr_to_var[marg]
        
        # get combined Z
        Z = _concat_ztrain_ztest(ztrain, ztest, tbin_train, tbin_test)
        Z = {marg:Z} # HACK
        
        # indvar_to_color_by = 0
        for var_color_by in effect_vars:
            if SAME_AXES_ACROSS_MARGS:
                val_minmax = [zmin, zmax]
            else:
                val_minmax = None
            fig, axes = plot_statespace_2d_overlaying_all_othervar(Z, marg, var_color_by, params_dpca, val_minmax, return_axes=True)
            # for ax in axes.flatten():
            #     ax.set_title()

            savefig(fig, f"{savedir}/2D-marg_{marg}-br_{br}-color_by_{var_color_by}-share_axes_{SAME_AXES_ACROSS_MARGS}.pdf")
            plt.close("all")
    

#### BETTER PLOTS, seaprate plots for train and test.
# First, collect all trajectories into dataframe, split up where each row is a single condition, with z shape (ndim, ntrials, ntimes), where ntrials is 1

res = []
for train_test in ["train", "test"]:
    if train_test=="train":
        DFRES_THIS = DFRES_TRAIN
    elif train_test=="test":
        DFRES_THIS = DFRES_TEST
    else:
        assert False
    for _, row in DFRES_THIS.iterrows():
        for marg, Zthis in row["Z"].items():
            # Collect data across effect vars
            for grp, idx in map_grp_to_idx.items():    
                z = Zthis[:, idx[0], idx[1], :] # (ndim, ntime)
                z = z.reshape(z.shape[0], 1, z.shape[1])
                
                # Collect data
                dat = {}
                dat["train_test"] = train_test
                dat["marg"] = marg
                for var, i in map_var_to_idx.items():
                    dat[var] = grp[i]    
                dat["bregion"] = row["bregion"]
                dat["which_level"] = row["which_level"]
                dat["event"] = row["event"]
                dat["twind"] = row["twind"]
                dat["explained_var"] = row["explained_var"]
                dat["params_dpca"] = row["params_dpca"]
                dat["times"] = row["params_dpca"]["times"]
                dat["z"] = z
                res.append(dat)               
    
DFRES_TRAINTEST = pd.DataFrame(res)

### GOOD PLOT separately train vs. test, each doing both scalar and traj,
# plotted on same space.
list_tbin_test = [0, -1], # each does separate set of plots...
### PLot params
times_to_mark = [0]
times_to_mark_markers = ["d"]
# text_plot_pt1 = "on"
text_plot_pt1 = None
markersize=7
marker="P"
time_bin_size = 0.05

for tbin_test in list_tbin_test:

    # Taske a single scalar time bin, differently for train and test
    list_z_scalar = []
    for i, row in DFRES_TRAINTEST.iterrows():
        z = row["z"] # (ndims, ntrials, ntimes)
        if row["train_test"]=="train":
            tbin = tbin_train
        elif row["train_test"]=="test":
            tbin = tbin_test
        else:
            assert False
        list_z_scalar.append(z[:,:,tbin][:,:,None]) # (ndims, ntrials, 1)
        
    DFRES_TRAINTEST["z_scalar"] = list_z_scalar
        
    
    # SCALAR PLOTS: train vs. test
    var_subplots = "train_test"
    var_color_by = "seqc_0_shape"
    # for traj_or_scalar in ["traj", "scalar"]:
    for traj_or_scalar in ["scalar"]:
        for marg in margs:
            for bregion in list_br:
                    
                if bregion=="vlPFC":
                    # Pick out specific df
                    b = DFRES_TRAINTEST["marg"]==marg
                    c = DFRES_TRAINTEST["bregion"]==bregion
                    dfthis = DFRES_TRAINTEST[b & c]
                    
                    
                    from neuralmonkey.analyses.state_space_good import trajgood_plot_colorby_splotby
                    fig, axes = trajgood_plot_colorby_splotby(dfthis, var_color_by, var_subplots, dims=(0,1),
                                                      traj_or_scalar=traj_or_scalar, mean_over_trials=True,
                                                      times_to_mark=times_to_mark,
                                                       times_to_mark_markers=times_to_mark_markers,
                                                       time_bin_size=time_bin_size,
                                                       markersize=markersize, marker=marker,
                                                       text_plot_pt1=text_plot_pt1,
                                                       alpha=0.2)
                    
                    savefig(fig, f"{savedir}/2D_train_vs_test-{traj_or_scalar}-marg_{marg}-{bregion}-var_{var_color_by}-ovar_{var_subplots}-tbin_test_{tbin_test}.pdf")
                    plt.close("all")


# Compute significance, based on decoder

The 1st mixing component looks merely like noise. But to be sure, we can run a significance analysis:

In [ ]:
significance_masks = dpca.significance_analysis(R,  trialR, n_shuffles=10, n_splits=10, n_consecutive=10)

In [ ]:
significance_masks

We can highlight the significant parts of the demixed components with a black bar underneath. Note that there is no significant analysis time, since there are no classes to compute the significance over.

In [ ]:
significance_masks["st"]

In [ ]:
Z["s"].shape

In [ ]:
time = arange(T)

figure(figsize=(16,7))
subplot(131)

for s in range(S):
    plot(time,Z['t'][0,s])

title('1st time component')
    
subplot(132)

for s in range(S):
    plot(time,Z['s'][0,s])

imshow(significance_masks['s'][0][None,:],extent=[0,250,amin(Z['s'])-1,amin(Z['s'])-0.5],aspect='auto',cmap='gray_r',vmin=0,vmax=1)
ylim([amin(Z['s'])-1,amax(Z['s'])+1])

title('1st stimulus component')
    
subplot(133)

for s in range(S):
    plot(time,Z['st'][0,s])

dZ = amax(Z['st'])-amin(Z['st'])
imshow(significance_masks['st'][0][None,:],extent=[0,250,amin(Z['st'])-dZ/10.,amin(Z['st'])-dZ/5.],aspect='auto',cmap='gray_r',vmin=0,vmax=1)
ylim([amin(Z['st'])-dZ/10.,amax(Z['st'])+dZ/10.])
    
title('1st mixing component')
show()